In [ ]:
using JuMP
using Gurobi
using LinearAlgebra
using Distributions
using Random
using PDMats

Random.seed!(42)

In [ ]:
using Revise

includet("../models/forward.jl")
import .Forward as Forward

includet("../models/inversedemand.jl")
import .InverseDemand as IODemand

includet("../models/inverselinreg.jl")
import .InverseLinReg as IOLinReg

includet("../datagen/data-generation.jl")
import .DataGeneration as DataGen


In [ ]:
function forward_example_params()::Forward.Params
    enabled_flows = ones(Bool, (4, 2))
    enabled_flows = [
        1 0;
        0 1;
        1 0;
        0 1
    ]

    return Forward.Params(
        n_paths=4, 
        n_commodities=2,
        capacities=[10, 15, 1000, 1000],
        design_costs=[10, 10, 100, 100],
        flow_costs=[10 0 ; 0 10 ; 100 0; 0 100],
        enabled_flows=enabled_flows
    )
end

forward_example_params()

In [ ]:
struct DataGenParams
    n_commodities::Integer
    n_features::Integer

    weights::Matrix
    noise_variance::Union{AbstractVector, Nothing}

    function DataGenParams(; weights::Matrix, noise_variance=nothing::Union{AbstractVector, Nothing})
        n_commodities, n_features = size(weights)
        
        if (noise_variance !== nothing && size(noise_variance)[1] != n_commodities)
            error("Invalid size $(size(noise_variance)) of noise variance, should be $(n_commodities)")
        end

        new(n_commodities, n_features, weights, noise_variance)
    end
end

dgen_params = DataGenParams(
    weights=[1.5 3 2 ; 1 2.5 -0.4], 
    noise_variance=[2.5, 1.2])

In [ ]:
n_points = 10

In [ ]:
function generate_input_features(datagen_params::DataGenParams, n_points; lower_bound=0, upper_bound=10)
    n_features = datagen_params.n_features
    
    distribution = Uniform.(fill(lower_bound, n_features), fill(upper_bound, n_features))
    mv_distribution = Product(distribution)

    return rand(mv_distribution, n_points)
end

features = generate_input_features(dgen_params, n_points)

In [ ]:
function generate_noise(datagen_params::DataGenParams)
    if datagen_params.noise_variance === nothing
        return zeros(datagen_params.n_commodities)
    end

    covariance_matrix = PDiagMat(datagen_params.noise_variance)
    distribution = DiagNormal(zeros(datagen_params.n_commodities), covariance_matrix)

    return vec(rand(distribution, 1))
end

function generate_noises(datagen_params::DataGenParams, n_points)
    return hcat([generate_noise(datagen_params) for _ in 1:n_points]...)
end

noises = generate_noises(dgen_params, n_points)

In [ ]:
function generate_demands(datagen_params::DataGenParams, features, noises)
    function predict_demand(features)        
        return datagen_params.weights * features
    end
    
    demands = mapslices(predict_demand, features, dims=1)
    noisy_demands = demands .+ noises
    non_negative_demands = max.(0, demands)

    return non_negative_demands
end

demands = generate_demands(dgen_params, features, noises)

In [ ]:
function generate_problem_params(base_params::IOLinReg.Params, demands; close_multiplier=1.1, far_multiplier=10, close_commodities=Set([1]))
    function create_capacities(demand)
        base_capacities = copy(base_params.forward_params.capacities)
        
        for (i, d) in enumerate(demand)
            new_capacity = (i in close_commodities) ? close_multiplier * d : far_multiplier * d
            base_capacities[i] = new_capacity
        end
        
        return base_capacities
    end

    new_capacities = create_capacities.(eachcol(demands))
    new_params = (IOLinReg.Params(new_capacity, base_params) for new_capacity in new_capacities)
    return collect(new_params)
end

base_prob_params = IOLinReg.Params(n_features=dgen_params.n_features, forward_params=forward_example_params(), with_noise=true)
problem_params = generate_problem_params(base_prob_params, demands)

In [ ]:
function generate_solution_points(datagen_params::DataGenParams, features, demands, problem_params; gurobi_env=nothing)
    create_and_solve_problem = (demand, param) ->
        Forward.create_and_solve_problem(param.forward_params, demand, silent=true, gurobi_env=gurobi_env)

    forward_sols = map(create_and_solve_problem, eachcol(demands), problem_params)
    solutions = map(IOLinReg.SolutionPoint, forward_sols, eachcol(features), eachcol(demands), problem_params)

    return solutions
end

g = Gurobi.Env()
solutions = generate_solution_points(dgen_params, features, demands, problem_params, gurobi_env=g)